# PRMT-2364 Generate EMIS conversation IDs for the last 2 weeks

In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime
import paths
from data.practice_metadata import read_asid_metadata

In [2]:
asid_lookup=read_asid_metadata("prm-gp2gp-ods-metadata-prod", "v2/2021/8/organisationMetadata.json")

transfers_raw = pd.read_parquet("s3://prm-gp2gp-transfer-data-prod/v4/2021/8/transfers.parquet")

transfers = transfers_raw\
    .join(asid_lookup.add_prefix("requesting_"), on="requesting_practice_asid", how="left")\
    .join(asid_lookup.add_prefix("sending_"), on="sending_practice_asid", how="left")\

last_week_of_august_bool = transfers["date_requested"] > datetime(2021, 8, 29)
last_week_of_august_transfers = transfers[last_week_of_august_bool]

In [3]:
emis_to_emis_bool = (last_week_of_august_transfers['sending_supplier'] == 'EMIS') & (last_week_of_august_transfers['requesting_supplier']=='EMIS')
emis_to_emis_transfers = last_week_of_august_transfers[emis_to_emis_bool]

technical_failure_bool = emis_to_emis_transfers["status"]=="TECHNICAL_FAILURE"
columns_of_interest = ["conversation_id", "requesting_supplier", "sending_supplier", "requesting_practice_ods_code", "sending_practice_ods_code", "sender_error_codes", "final_error_codes", "intermediate_error_codes", "status", "failure_reason", "date_requested", "date_completed"]

### EMIS --> EMIS, no error code, COPC(s) not Acknowledged

In [4]:
copc_not_acknowledged_bool = emis_to_emis_transfers['failure_reason'] == 'COPC(s) not Acknowledged'
copc_not_acknowledged_sample = emis_to_emis_transfers[copc_not_acknowledged_bool & technical_failure_bool].sample(5)
copc_not_acknowledged_sample[columns_of_interest]

,conversation_id,requesting_supplier,sending_supplier,requesting_practice_ods_code,sending_practice_ods_code,sender_error_codes,final_error_codes,intermediate_error_codes,status,failure_reason,date_requested,date_completed
104461,346AACCA-DDF7-4D7F-A3A7-CFA5A9D60C01,EMIS,EMIS,H81010,H81043,[nan],[],[],TECHNICAL_FAILURE,COPC(s) not Acknowledged,2021-08-31 14:05:04.731,NaT
104121,710B03C1-325B-43A7-9A96-BA8912A4D3B8,EMIS,EMIS,M86605,M86034,[nan],[],[],TECHNICAL_FAILURE,COPC(s) not Acknowledged,2021-08-31 15:52:34.847,NaT
109438,C78BDDE3-4D21-4E06-91CB-700664604988,EMIS,EMIS,P81002,P81112,[nan],[],[],TECHNICAL_FAILURE,COPC(s) not Acknowledged,2021-08-31 12:31:00.029,NaT
112025,3C3B2B20-2791-44A5-96CF-1E3C11632BD6,EMIS,EMIS,G82799,Y02506,[nan],[],[],TECHNICAL_FAILURE,COPC(s) not Acknowledged,2021-08-31 11:18:53.055,NaT
108947,1B9FA09A-7BFA-438A-AE21-435C15395FE6,EMIS,EMIS,P81622,P81683,[nan],[],[],TECHNICAL_FAILURE,COPC(s) not Acknowledged,2021-08-31 12:36:50.132,NaT


### EMIS --> EMIS, error code 25 (Large messages rejected due to timeout duration reached of overall transfer)

In [5]:
error_code_25_bool = emis_to_emis_transfers["final_error_codes"].apply(lambda error_codes: 25 in error_codes)
error_code_25_sample = emis_to_emis_transfers[error_code_25_bool & technical_failure_bool].sample(5)
error_code_25_sample[columns_of_interest]

,conversation_id,requesting_supplier,sending_supplier,requesting_practice_ods_code,sending_practice_ods_code,sender_error_codes,final_error_codes,intermediate_error_codes,status,failure_reason,date_requested,date_completed
112559,39D89185-BEC9-44B5-A261-D27CC787EDFE,EMIS,EMIS,F82055,F82002,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-08-31 11:31:48.465,2021-09-02 09:48:14.765
105498,2F24FD43-E635-441A-9548-FF7F7AE051EF,EMIS,EMIS,L83049,L83014,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-08-31 16:06:25.325,2021-09-09 07:02:11.242
113936,FAD68DAE-63AF-4631-978B-545059CDC00A,EMIS,EMIS,H85112,H85029,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-08-31 10:17:27.042,2021-09-13 07:25:30.816
110404,1D030A1B-EA29-49C4-9092-AB83084F54EB,EMIS,EMIS,P88623,P84068,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-08-31 11:12:57.230,2021-09-14 08:25:15.173
111009,4DE9A75F-8D63-4970-B325-3057BA83446E,EMIS,EMIS,G81039,P88017,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-08-31 08:05:46.160,2021-09-08 07:12:44.036


### EMIS --> EMIS, error code 20 (Spine system responded with an error)

In [6]:
error_code_20_bool = emis_to_emis_transfers["sender_error_codes"].apply(lambda error_codes: 20 in error_codes)
error_code_20_sample = emis_to_emis_transfers[error_code_20_bool & technical_failure_bool].sample(5)
error_code_20_sample[columns_of_interest]

,conversation_id,requesting_supplier,sending_supplier,requesting_practice_ods_code,sending_practice_ods_code,sender_error_codes,final_error_codes,intermediate_error_codes,status,failure_reason,date_requested,date_completed
121724,CD6323F3-C44B-44C2-998E-625E3127694A,EMIS,EMIS,H83031,H85653,[20.0],[],[],TECHNICAL_FAILURE,Core Extract not Sent,2021-08-31 08:33:19.504,NaT
104051,674C5CF7-981F-4084-8285-23AACE3B3992,EMIS,EMIS,K84041,K84023,[20.0],[],[],TECHNICAL_FAILURE,Core Extract not Sent,2021-08-31 16:28:10.236,NaT
114740,5BC8A776-D569-4840-A217-91EA285E864C,EMIS,EMIS,P81073,P81016,[20.0],[],[],TECHNICAL_FAILURE,Core Extract not Sent,2021-08-31 10:29:53.555,NaT
113113,AA2FFDA1-B459-4B63-8FCD-32A7000BE1EC,EMIS,EMIS,N81019,N81075,[20.0],[],[],TECHNICAL_FAILURE,Core Extract not Sent,2021-08-31 10:59:37.639,NaT
115677,065E47A6-B6ED-4327-9388-13D16593D4E9,EMIS,EMIS,F86616,F86627,[20.0],[],[],TECHNICAL_FAILURE,Core Extract not Sent,2021-08-31 11:30:22.804,NaT


### EMIS --> EMIS, error code 19 (Sender check indicates that Requester is not the patient’s current healthcare provider)

In [7]:
error_code_19_bool = emis_to_emis_transfers["sender_error_codes"].apply(lambda error_codes: 19 in error_codes)
error_code_19_sample = emis_to_emis_transfers[error_code_19_bool & technical_failure_bool].sample(5)
error_code_19_sample[columns_of_interest]

,conversation_id,requesting_supplier,sending_supplier,requesting_practice_ods_code,sending_practice_ods_code,sender_error_codes,final_error_codes,intermediate_error_codes,status,failure_reason,date_requested,date_completed
109454,21C548CE-120F-4CF8-A25C-E3167D1E98D7,EMIS,EMIS,N83628,M85025,[19.0],[],[],TECHNICAL_FAILURE,Core Extract not Sent,2021-08-31 12:22:37.047,NaT
111717,2421F095-1F0E-4A41-ACE0-1D7145EBB147,EMIS,EMIS,NaN,C84714,[19.0],[],[],TECHNICAL_FAILURE,Core Extract not Sent,2021-08-31 14:24:08.638,NaT
112493,B4CD79A3-6877-4966-9253-C995A00F7B05,EMIS,EMIS,L81031,L81009,[19.0],[],[],TECHNICAL_FAILURE,Core Extract not Sent,2021-08-31 12:51:39.141,NaT
108046,59650E9F-E3DB-4ECC-BE49-8D3F6B062A32,EMIS,EMIS,H83017,G85695,[19.0],[],[],TECHNICAL_FAILURE,Core Extract not Sent,2021-08-31 14:18:30.837,NaT
123830,E7B7A4BC-45EE-4F53-8D92-272E456B9A46,EMIS,EMIS,Y03023,F84044,[19.0],[],[],TECHNICAL_FAILURE,Core Extract not Sent,2021-08-29 14:59:05.318,NaT


### EMIS --> SystmOne, error code 31 (The overall EHR Extract has been rejected because one or more attachments via Large Messages were not received)

In [8]:
emis_to_tpp_bool = (last_week_of_august_transfers['sending_supplier'] == 'EMIS') & (last_week_of_august_transfers['requesting_supplier']=='SystmOne')
emis_to_tpp_transfers = last_week_of_august_transfers[emis_to_tpp_bool]

error_code_31_bool = emis_to_tpp_transfers["final_error_codes"].apply(lambda error_codes: 31 in error_codes)
error_code_31_sample = emis_to_tpp_transfers[error_code_31_bool].sample(5)
error_code_31_sample[columns_of_interest]

,conversation_id,requesting_supplier,sending_supplier,requesting_practice_ods_code,sending_practice_ods_code,sender_error_codes,final_error_codes,intermediate_error_codes,status,failure_reason,date_requested,date_completed
104160,BAD22840-0A71-11EC-AC71-4756B0A0F01C,SystmOne,EMIS,A83045,L82029,[nan],[31.0],[],TECHNICAL_FAILURE,Final Error,2021-08-31 15:40:12.829,2021-09-08 01:51:30.783
115387,54AC0D40-0A3C-11EC-A0A8-3BA054622A39,SystmOne,EMIS,C88023,M87601,[nan],[31.0],[],TECHNICAL_FAILURE,Final Error,2021-08-31 09:17:58.257,2021-09-05 04:29:37.752
105435,1FB315B0-0A7E-11EC-88FF-CDFE43D01593,SystmOne,EMIS,J81033,K84082,[nan],[31.0],[],TECHNICAL_FAILURE,Final Error,2021-08-31 17:08:57.421,2021-09-04 01:20:14.823
110584,0CD273B0-0A46-11EC-9849-09931064B509,SystmOne,EMIS,D83012,F81053,[nan],[31.0],[],TECHNICAL_FAILURE,Final Error,2021-08-31 10:27:32.495,2021-09-02 21:38:30.568
113201,5C489B90-0A4B-11EC-8E39-57DD4F99727A,SystmOne,EMIS,E87609,F85014,[nan],[31.0],[],TECHNICAL_FAILURE,Final Error,2021-08-31 11:05:33.135,2021-09-01 04:16:06.558


### SystmOne --> EMIS, error code 30 (Large message general failure)

In [9]:
tpp_to_emis_bool = (last_week_of_august_transfers['sending_supplier'] == 'SystmOne') & (last_week_of_august_transfers['requesting_supplier']=='EMIS')
tpp_to_emis_transfers = last_week_of_august_transfers[tpp_to_emis_bool]

error_code_30_bool = tpp_to_emis_transfers["sender_error_codes"].apply(lambda error_codes: 30 in error_codes)
error_code_30_sample = tpp_to_emis_transfers[error_code_30_bool].sample(5)
error_code_30_sample[columns_of_interest]

,conversation_id,requesting_supplier,sending_supplier,requesting_practice_ods_code,sending_practice_ods_code,sender_error_codes,final_error_codes,intermediate_error_codes,status,failure_reason,date_requested,date_completed
113812,EA9F4110-6CFE-440D-9984-BDBD62F4DDE0,EMIS,SystmOne,M85077,C81042,[30.0],[],[],TECHNICAL_FAILURE,Contains Fatal Sender Error,2021-08-31 10:20:43.053,NaT
111921,753B30B2-8D99-430A-843B-ED370184E6B4,EMIS,SystmOne,G81102,G81086,[30.0],[],[],TECHNICAL_FAILURE,Contains Fatal Sender Error,2021-08-31 13:26:26.064,NaT
122669,0A1CBB99-D081-47CC-89F1-47FBF48F17C6,EMIS,SystmOne,M85624,C82024,[30.0],[],[],TECHNICAL_FAILURE,Contains Fatal Sender Error,2021-08-31 09:16:49.459,NaT
114337,909359B4-0EE6-41DC-A675-B5FB75F28E8C,EMIS,SystmOne,H82045,M85002,[30.0],[],[],TECHNICAL_FAILURE,Contains Fatal Sender Error,2021-08-31 10:14:55.627,NaT
114720,BECB3F8A-E8BD-4C64-8002-E8D84AC6E147,EMIS,SystmOne,P84639,C81029,[30.0],[],[],TECHNICAL_FAILURE,Contains Fatal Sender Error,2021-08-31 11:14:16.755,NaT
